### 1) Mineria de Datos sobre Redes Sociales y Sitios Web
#### 1.1)   Tomando datos de la red social Twitter, al menos 200 tweets, realice un script que recolecte, procese y visualice los datos con las siguientes restricciones:

##### a)Los tres trends más RT del momento.

In [66]:
#Cargamos los paquetes necesarios para entrar a la API de Twitter
import twitter
import json
from prettytable import PrettyTable
from collections import Counter
import pandas as pd

keys = pd.read_json('keys.json')['KEYS']
CONSUMER_KEY = keys['CONSUMER_KEY']
CONSUMER_SECRET = keys['CONSUMER_SECRET']
OAUTH_TOKEN = keys['OAUTH_TOKEN']
OAUTH_TOKEN_SECRET = keys['OAUTH_TOKEN_SECRET']

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
twitter_api = twitter.Twitter(auth=auth)

#### Seteamos los WOEID para obtener los resultados 

In [67]:
SPAIN_WOE_ID = 23424950
spain_trends = twitter_api.trends.place(_id=SPAIN_WOE_ID)
hashtags = spain_trends[0]['trends']
trends = map(lambda i: hashtags[i]['name'], range(len(hashtags)))[:3]
pt = PrettyTable()
pt.add_column('Spain Trends', trends)
print pt

+-----------------+
|   Spain Trends  |
+-----------------+
|    #GHDirecto   |
| #ToroDeLaVegaEP |
|   #meinteresa   |
+-----------------+


#### b )   Listar nombres de usuarios que publicaron con los hashtag de los trends del punto anterior.

In [68]:
# Obtenemos los tweets
CANT = 10
in_spain = set(trends)

tweets = []
for i in range(len(in_spain)):
    results = twitter_api.search.tweets(q=in_spain.pop() , count=CANT)
    tweets += results['statuses']

# Obtenemos los nombres de los users
screen_name = map( lambda i: tweets[i]['user']['screen_name'], range(len(tweets)))
screen_name = list(set(screen_name)) #sin repetidos
pt = PrettyTable()
pt.add_column('Users', screen_name )

#mostramos los resultados en una tabla
print pt

+-----------------+
|      Users      |
+-----------------+
|  AnuskaNewYork  |
|  MarianoRapJoy  |
|     lolasnga    |
|  ComentaFamosos |
|     geanny83    |
|    fandelat3    |
|      atmacg     |
|     JaskiATS    |
|  caprichoreal15 |
|  NachoBassPerez |
|   marsolina56   |
|      fanega     |
|    DharmaValo   |
|   DannyAndara   |
|  alexmaria9906  |
| carmen_maria_19 |
|   lorexana106   |
|    JSACROSOFT   |
|    bentorivan   |
|   haztedenotar  |
| CFMariahb_Myhyv |
| milagrospenagos |
| GurruGHChrislet |
|    LenaLenama   |
|   Jaime_lacon   |
|    paquialma    |
|  isidrotoledo1  |
+-----------------+


#### c) Cuales son las cinco palabras más utilizadas en los tweets del primer item.

In [69]:
text = map( lambda i: tweets[i]['text'], range(len(tweets)))
words = [ word for twt in text for word in twt.split() ]
c = Counter(words)
#print c.most_common()

pt = PrettyTable(field_names=['Words', 'Count'])
map(lambda r: pt.add_row(r), c.most_common()[:5])
print pt

+-------+-------+
| Words | Count |
+-------+-------+
|   RT  |   19  |
|   el  |   14  |
|  que  |   12  |
|   de  |   12  |
|   a   |   11  |
+-------+-------+


#### d) Listar los primeros 10 usuarios con mayor cantidad de seguidores

In [70]:
lista_users_followers = map( lambda i: (tweets[i]['user']['screen_name'],tweets[i]['user']['followers_count']), range(len(tweets)))

lista_users_followers = sorted(lista_users_followers, key=lambda x: x[1], reverse=True)[:10]
users = map((lambda x: x[0]), lista_users_followers)
followers = map((lambda x: x[1]), lista_users_followers)


pt = PrettyTable()
pt.add_column('screen_name', users)
pt.add_column('followers_count', followers)
print pt

+-----------------+-----------------+
|   screen_name   | followers_count |
+-----------------+-----------------+
|      fanega     |       2929      |
|      fanega     |       2929      |
| CFMariahb_Myhyv |       2618      |
|    JSACROSOFT   |       2328      |
|     geanny83    |       2169      |
|   lorexana106   |       1963      |
| carmen_maria_19 |       1378      |
|     lolasnga    |       1007      |
|    fandelat3    |       839       |
|  MarianoRapJoy  |       824       |
+-----------------+-----------------+


#### e) Listar la ubicación (o en su defecto time-zone ) del tweet.

In [71]:
users = map( lambda i: tweets[i]['user']['screen_name'], range(len(tweets)))
location_or_tz = map( lambda i: tweets[i]['user']['location'] or tweets[i]['user']['time_zone'], range(len(tweets)))

pt = PrettyTable()
pt.add_column('User', users)
pt.add_column('Location or TZ', location_or_tz)
print pt

+-----------------+--------------------------------+
|       User      |         Location or TZ         |
+-----------------+--------------------------------+
| milagrospenagos |              None              |
|   DannyAndara   |    ÜT: 10.496789,-66.846729    |
|   haztedenotar  |              None              |
|  isidrotoledo1  |         GANDIA ESPAÑA          |
|    bentorivan   |   Las Palmas de Gran Canaria   |
|    paquialma    |              None              |
|    LenaLenama   |            cordoba             |
| carmen_maria_19 |   Pacific Time (US & Canada)   |
|  MarianoRapJoy  |           Españistan           |
|    DharmaValo   |             Málaga             |
|  AnuskaNewYork  |             España             |
| GurruGHChrislet |             Gh 16              |
|     JaskiATS    |            #Enerski            |
|   lorexana106   | https://www.facebook.com/friki |
|   marsolina56   |       Bilbao, País Vasco       |
| CFMariahb_Myhyv |           Casablanca      

#### f) Listar los cinco tweets más populares.


In [94]:
texto_y_rt = map( lambda i: (tweets[i]['text'], tweets[i]['retweet_count'], tweets[i]['user']['id']), range(len(tweets)))

texto_y_rt = sorted(texto_y_rt, key=lambda x: x[1], reverse=True)[:5]
popular_tw = texto_y_rt[0]
texto = map((lambda x: x[0]), texto_y_rt)
rts = map((lambda x: x[1]), texto_y_rt)


pt = PrettyTable()
pt.add_column('Tweet', texto)
pt.max_width['Tweet'] = 80
pt.add_column('Retweets', rts)
pt.align = 'l'
print pt

+----------------------------------------------------------------------------------+----------+
| Tweet                                                                            | Retweets |
+----------------------------------------------------------------------------------+----------+
| RT @subversivos_: Y este es el nivel intelectual del Jefe de la tribu de         | 449      |
| #Tordesillas el que permite la salvajada                                         |          |
| #ToroDeLaVegaEP http://t.…                                                       |          |
| RT @EsppeonzAguirre: Los de Tordesillas descubren un nuevo teorema de Pitágoras. | 103      |
| Descubre quiénes son los catetos... #ToroDeLaVegaEP http:/…                      |          |
| RT @salvameoficial: ¿Queréis que hablemos ya de Gran Hermano y nos dejemos de    | 97       |
| historias?                                                                       |          |
| RT=SÍ, VAMOS                          

#### g) Listar a los seguidores del autor del tweet más popular

In [109]:
import tweepy
auth_tweepy = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth_tweepy.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

api = tweepy.API(auth_tweepy)
user = api.get_user(popular_tw[2])
title = "%s's followers:" % (user.screen_name)
followers = map((lambda f: f.screen_name), user.followers(count=100))

pt = PrettyTable()
pt.add_column(title, followers)
print pt


+-------------------------+
| JSACROSOFT's followers: |
+-------------------------+
|         DobleVB         |
|        fsatriany2       |
|     TupaCatedralBqt     |
|     1874acc9d05241e     |
|      gabriela22ann      |
|     miralmiral13482     |
|      jorgeamoralesv     |
|        lendl_gta        |
|       marco_amc89       |
|         lemeza47        |
|       BalcazarMama      |
|      hoychavezvive      |
|      jesusherberts      |
|         dblancog        |
|         cerebro5        |
|       NormelysVzla      |
|        bucareroja       |
|      lagosriederer      |
|        rojasjj72        |
|         Emilkacb        |
|     KarenMikesmall1     |
|       RojoYClaro1       |
|       AherzAntonio      |
|       ines_guayaca      |
|         DDu2015         |
|      planetarojotv      |
|       wperezyanes       |
|      NildaGalarraga     |
|         karel259        |
|      Vallero33oo07      |
|      Marlene_Rojas      |
|       LuzMarinaft       |
|       Rockdirecto 